# Colombian Constitution EDA.
    Brief resume about the methodology and the results that we got.

## About the Document
    The Political Constitution of the Republic of Colombia of 1991 is the Magna Carta of the Republic of Colombia. It was promulgated in the Constitutional Gazette number 114 of Thursday, July 4, 1991, and is also known as the Constitution of Human Rights. It replaced the Political Constitution of 1886 and was issued during the presidency of the liberal César Gaviria.1

## DS Pipeline



## Document Data Model

PowerPoint graph, meanwhile we can use the hierarchy of the articles dictionary

```
hierarchy = {
    'TITULO' : 'h1',
    'DISPOSICIONES' : 'h1',
    'CAPITULO' : 'h2',
    'ARTÍCULO' : 'p'

}
```

## EDA

### Libraries 

In [1]:
import numpy as np
import pandas as pd
# For visualizations
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
# import tensorflow  as tf
# # tf.disable_v2_behavior()
# import tensorflow_hub as hub
# import tensorflow_text
import os 
from tf_model import embed_text


In [10]:
root_folder = "LegalSearcher/ReadFiles"
constitution_f = f'{root_folder}/constitucion-embedding.json'
filepath = os.path.abspath(os.path.join('','..', '..', constitution_f))
#import Dataset
df_cons = pd.read_json(filepath)
df_cons

,index,legal_source,id,book,headline,chapter,section,part,article,dot_comma_sep
0,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000001,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 1.', 'content': ['Artículo ...","[Artículo 1., Colombia es un Estado social de ..."
1,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000002,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 2.', 'content': ['Artículo ...","[Artículo 2., Son fines esenciales del Estado:..."
2,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000003,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 3.', 'content': ['Artículo ...","[Artículo 3., La soberanía reside exclusivamen..."
3,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000004,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 4.', 'content': ['Artículo ...","[Artículo 4., La Constitución es norma de norm..."
4,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000005,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 5.', 'content': ['Artículo ...","[Artículo 5., El Estado reconoce, sin discrimi..."
...,...,...,...,...,...,...,...,...,...,...
435,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00001408000436,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': 'CAPITULO 8 ', 'name': ' '}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo transitorio 56.', 'content'...","[Artículo transitorio 56., Mientras se expide ..."
436,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00001408000437,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': 'CAPITULO 8 ', 'name': ' '}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo transitorio 57.', 'content'...","[Artículo transitorio 57., El Gobierno formará..."
437,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00001408000438,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': 'CAPITULO 8 ', 'name': ' '}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo transitorio 58.', 'content'...","[Artículo transitorio 58., Autorizase al Gobie..."
438,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00001408000439,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': 'CAPITULO 8 ', 'name': ' '}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo transitorio 59.', 'content'...","[Artículo transitorio 59., La presente constit..."


In [3]:
df_cons=df_cons[['id','dot_comma_sep']]
df_cons.replace(r'\n','', regex=True,inplace=True) 
df_cons['dot_comma_sep'] = df_cons['dot_comma_sep'].apply(lambda x:x[1:])
# df_cons.rename(columns={'dot_comma_sep':'dot_comma_sep'},inplace=True)
df_cons.head()


/home/maverick/GitHub/.wslenv1/lib/python3.8/site-packages/pandas/core/frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-3-5d537814288a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cons['dot_comma_sep'] = df_cons['dot_comma_sep'].apply(lambda x:x[1:])


,id,dot_comma_sep
0,constitucion00000100000001,"[Colombia es un Estado social de derecho, orga..."
1,constitucion00000100000002,[Son fines esenciales del Estado: servir a la ...
2,constitucion00000100000003,[La soberanía reside exclusivamente en el pueb...
3,constitucion00000100000004,"[La Constitución es norma de normas., En todo ..."
4,constitucion00000100000005,"[El Estado reconoce, sin discriminación alguna..."


In [4]:


codes_folder = "../ReadFiles/Embeddings"
codes_path = os.path.abspath(codes_folder)
toLoad_entries = os.listdir(codes_path)
df = 0
total = len(toLoad_entries)

main_df = None
col_delimeter = ' = | = '
for entry in toLoad_entries:
    df += 1
    # print(df, 'loading:', entry, end=col_delimeter)

    entry_path = os.path.join(codes_folder, entry)
    new_df = pd.read_json(entry_path)
    # print('DF shape:', new_df.shape, end=col_delimeter)
    new_df = new_df[['id','dot_comma_sep']]
    
    if df == 1:
        main_df = new_df
    else:
        main_df = main_df.append(new_df, ignore_index=True)
        # print('NEW DF shape:', main_df.shape)

main_df

,id,dot_comma_sep
0,prope00000100000001,"[Artículo 1º., Dignidad humana., Los intervini..."
1,prope00000100000002,"[Artículo 2º., Libertad., Modificado por el a..."
2,prope00000100000003,"[Artículo 3º., Prelación de los tratados inter..."
3,prope00000100000004,"[Artículo 4º., Igualdad., Es obligación de los..."
4,prope00000100000005,"[Artículo 5º., Imparcialidad., En ejercicio de..."
...,...,...
16282,trabajo00002301000654,"[ARTICULO 486., SANCIONES Y PROCEDIMIENTO., Lo..."
16283,trabajo00002301000655,"[ARTICULO 487., FUNCIONARIOS DE INSTRUCCION., ..."
16284,trabajo00002302000656,"[ARTICULO 488., REGLA GENERAL., Las acciones c..."
16285,trabajo00002302000657,"[ARTICULO 489., INTERRUPCION DE LA PRESCRIPCIO..."


In [11]:
###### Sentence Encoder for DataFrame
df_cons['embedds'] = df_cons['dot_comma_sep'].apply(lambda x:embed_text(x))
df_cons.head()

,index,legal_source,id,book,headline,chapter,section,part,article,dot_comma_sep,embedds
0,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000001,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 1.', 'content': ['Artículo ...","[Artículo 1., Colombia es un Estado social de ...","[0.004690078552812338, 0.07231230288743973, 0...."
1,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000002,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 2.', 'content': ['Artículo ...","[Artículo 2., Son fines esenciales del Estado:...","[0.0040732030756771564, 0.09081056714057922, 0..."
2,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000003,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 3.', 'content': ['Artículo ...","[Artículo 3., La soberanía reside exclusivamen...","[0.010173899121582508, 0.08467913419008255, 0...."
3,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000004,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 4.', 'content': ['Artículo ...","[Artículo 4., La Constitución es norma de norm...","[0.004502664320170879, 0.09044172614812851, 0...."
4,constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,constitucion00000100000005,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': None, 'name': None}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo 5.', 'content': ['Artículo ...","[Artículo 5., El Estado reconoce, sin discrimi...","[-0.03632644936442375, 0.058461349457502365, 0..."


In [7]:
###### Sentence Encoder for DataFrame
main_df['embedds'] = main_df['dot_comma_sep'].apply(lambda x:embed_text(x))
main_df.head()
# art1_res = embed(art1)

,id,dot_comma_sep,embedds
0,prope00000100000001,"[Artículo 1º., Dignidad humana., Los intervini...","[0.029562022536993027, 0.06292273849248886, 0...."
1,prope00000100000002,"[Artículo 2º., Libertad., Modificado por el a...","[0.01172316912561655, 0.09270897507667542, 0.0..."
2,prope00000100000003,"[Artículo 3º., Prelación de los tratados inter...","[0.03222179412841797, 0.08490251004695892, 0.0..."
3,prope00000100000004,"[Artículo 4º., Igualdad., Es obligación de los...","[0.024402596056461334, 0.09127292782068253, 0...."
4,prope00000100000005,"[Artículo 5º., Imparcialidad., En ejercicio de...","[-0.012061411514878273, 0.05749593302607536, 0..."


In [12]:
main_df = main_df.append(df_cons, ignore_index=True)
main_df

,id,dot_comma_sep,embedds,index,legal_source,book,headline,chapter,section,part,article
0,prope00000100000001,"[Artículo 1º., Dignidad humana., Los intervini...","[0.029562022536993027, 0.06292273849248886, 0....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,prope00000100000002,"[Artículo 2º., Libertad., Modificado por el a...","[0.01172316912561655, 0.09270897507667542, 0.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,prope00000100000003,"[Artículo 3º., Prelación de los tratados inter...","[0.03222179412841797, 0.08490251004695892, 0.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,prope00000100000004,"[Artículo 4º., Igualdad., Es obligación de los...","[0.024402596056461334, 0.09127292782068253, 0....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,prope00000100000005,"[Artículo 5º., Imparcialidad., En ejercicio de...","[-0.012061411514878273, 0.05749593302607536, 0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16722,constitucion00001408000436,"[Artículo transitorio 56., Mientras se expide ...","[-0.0572219081223011, 0.007201745640486479, 0....",constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': 'CAPITULO 8 ', 'name': ' '}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo transitorio 56.', 'content'..."
16723,constitucion00001408000437,"[Artículo transitorio 57., El Gobierno formará...","[-0.03703949972987175, 0.009493881836533546, 0...",constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': 'CAPITULO 8 ', 'name': ' '}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo transitorio 57.', 'content'..."
16724,constitucion00001408000438,"[Artículo transitorio 58., Autorizase al Gobie...","[-0.056596074253320694, 0.023641904816031456, ...",constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': 'CAPITULO 8 ', 'name': ' '}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo transitorio 58.', 'content'..."
16725,constitucion00001408000439,"[Artículo transitorio 59., La presente constit...","[-0.05907650291919708, 0.02931896038353443, 0....",constitucion,CONSTITUCION POLITICA DE COLOMBIA\n,"{'title': None, 'name': None}","{'title': 'DISPOSICIONES TRANSITORIAS ', 'name...","{'title': 'CAPITULO 8 ', 'name': ' '}","{'title': None, 'name': None}","{'title': None, 'name': None}","{'name': 'Artículo transitorio 59.', 'content'..."


In [17]:
main_df.drop(['dot_comma_sep','index','legal_source','book','headline','chapter','section',	'part',	'article'],axis=1,inplace=True)

In [ ]:
query0 = embed_text(['Colombia es un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista, fundada en el respeto de la dignidad humana, en el trabajo y la solidaridad de las personas que la integran y en la prevalencia del interés general'])
query1 = embed_text(['organizacion de colombia'])
query2 = embed_text(['Fines esenciales del estado'])
query3 = embed_text(['Representantes del pueblo'])
query4 = embed_text(['derechos humanos'])
query5 = embed_text(['tratados y convenios'])
query6 = embed_text(['Lorem ipsum dolor sit amet'])
query7 = embed_text(['organos del poder publico'])
query8 = embed_text(['Propiedad Intelectual'])
query9 = embed_text(['cuidado del medio ambiente'])
query10 = embed_text(['Ley de 1992'])
query11 = embed_text(['pizza'])

In [ ]:
# Cosine Similarity Function
def similarity(A,B):
    cos_sim = np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))+1
    return cos_sim

In [ ]:
# Finding the most similar
B = query5
count = 0
res = []
for index, row in df.iterrows():
    cos_sim = similarity(row['embedds'],B)
    if  cos_sim > count:
        count = cos_sim
        res = [index, row['article_list'],count]
    elif count < 1.2:
        res = "Sorry parce we Couldn't find anything about that in the Constitution."
    
# print(f'Query ingresada: {B}')    
print(res)

In [20]:
from fastparquet import write 
write('Cons_codes.parq', main_df)